In [2]:
from transformers import AutoTokenizer 
from datasets import load_dataset
import csv



In [8]:
tokenizer = AutoTokenizer.from_pretrained("Cedille/fr-boris")

In [3]:
tokenizer("Salut les gens", return_tensors="pt")

{'input_ids': tensor([[19221,   315, 10287,   308,   641]]), 'attention_mask': tensor([[1, 1, 1, 1, 1]])}

In [5]:
datasets = load_dataset('csv', data_files='./attempt.csv')

Using custom data configuration default-33369dc380894e49


Extracting data files: 100%|██████████| 1/1 [00:00<00:00, 362.45it/s]
Generating train split: 0 examples [00:00, ? examples/s]/Users/naowak/Library/Python/3.9/lib/python/site-packages/datasets/download/streaming_download_manager.py:776: FutureWarning: the 'mangle_dupe_cols' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'mangle_dupe_cols'
  return pd.read_csv(xopen(filepath_or_buffer, "rb", use_auth_token=use_auth_token), **kwargs)


Dataset csv downloaded and prepared to /Users/naowak/.cache/huggingface/datasets/csv/default-33369dc380894e49/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317. Subsequent calls will reuse this data.


100%|██████████| 1/1 [00:00<00:00, 264.54it/s]


In [6]:
datasets['train']

Dataset({
    features: ['text'],
    num_rows: 2
})

In [16]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding=True)

tokenizer.pad_token = tokenizer.eos_token
tokenized_datasets = datasets.map(
    tokenize_function,
    batched=True,
    remove_columns=datasets["train"].column_names,
)

100%|██████████| 1/1 [00:00<00:00, 164.61ba/s]


In [19]:
tokenized_datasets['train'][0]['input_ids']

[34, 721, 72, 1556, 555, 424, 431, 527, 1221, 15573]

In [13]:
block_size = 1024

def group_texts(examples):


    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    print(concatenated_examples)
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
    # customize this part to your needs.
    total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i: i + block_size]
            for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
)

100%|██████████| 1/1 [00:00<00:00, 434.60ba/s]

{'input_ids': [34, 721, 72, 1556, 555, 424, 431, 527, 1221, 15573, 34, 5857, 277, 10924, 269, 72, 38078, 24685, 1769], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


In [4]:
with open('attempt.csv', mode='w', encoding='utf-8') as csv_file:
    fieldnames = ['text']
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerow({'text': "Ceci est un supeber discuer"})
    writer.writerow({'text': "Cette fois ci sans fautes"})

In [18]:
lm_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 4970
    })
})

In [ ]:
import csv

with open('train.txt', encoding='utf-8') as txtfile:
    all_text = txtfile.read()
with open('train.csv', mode='w', encoding='utf-8') as csv_file:
    fieldnames = ['text']
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerow({'text': all_text})